In [8]:
library(fixest)
library(dplyr)
library(readr)

In [9]:
base = read_csv('base_tpv_master.csv')

Rows: 1044274 Columns: 30
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (8): document, tpv_stone, share_tpv_debit, share_tpv_chip_present, sha...
dbl  (21): index, tpv_total, share_tpv_credit, average_ticket_size, average_...
date  (1): reference_month

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [11]:
head(base)

index,document,reference_month,tpv_stone,tpv_total,share_tpv_debit,share_tpv_credit,share_tpv_chip_present,share_tpv_chip_not_present,share_tpv_recurring,⋯,grupo_cnae_fiscal_principal,total_estabelecimentos,media_cnpjs_por_socio,qtd_socios_no_mes,log_total_tpv_credit,log_total_tpv_credit_mun,log_total_tpv_credit_lag,log_total_tpv_credit_pred,residuo,residuo_pct
<dbl>,<chr>,<date>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
5575,36664802000143,2025-01-01,4078.360000000,4078.36,0E-9,1.0000000,NA,NA,NA,⋯,43,68,1,1,8.313450,13.81223,9.689551,9.192460,-0.87900928,-0.58480595
6125,53473882000179,2025-01-01,76567.590000000,76567.59,0.417741000,0.5822590,1.000000000,0E-9,0E-9,⋯,56,4650,5,1,10.705089,17.98507,9.049367,9.124195,1.58089433,3.85929967
6541,11938709000159,2025-01-01,0E-9,38915.43,0.026931000,0.9730690,0.851671000,0.148329000,0E-9,⋯,47,22848,2,1,10.541846,18.75019,10.321683,10.177654,0.36419176,0.43935020
8631,53731288000130,2025-01-01,21407.200000000,21407.20,0.713675000,0.2863250,1.000000000,0E-9,0E-9,⋯,93,10452,2,1,8.720855,21.71596,8.906461,9.128394,-0.40753945,-0.33471480
13320,01059030000136,2025-01-01,7525.630000000,44179.41,0.548871000,0.4511290,1.000000000,0E-9,0E-9,⋯,56,8736,1,1,9.900012,19.08415,9.971964,9.817344,0.08266793,0.08618106
13668,16964695000161,2025-01-01,35965.330000000,35965.33,0.201118688,0.7988813,NA,NA,NA,⋯,75,470,3,1,10.265768,19.48203,11.339361,11.192539,-0.92677167,-0.60417048


In [19]:
base$municipio = factor(base$municipio)
base$grupo_cnae_fiscal_principal = factor(base$grupo_cnae_fiscal_principal)

In [23]:
base$uf = substr(base$municipio,1,2)

In [26]:
summary(feols(log_total_tpv_credit ~ log_total_tpv_credit_lag + log_total_tpv_credit_mun + media_cnpjs_por_socio + qtd_socios_no_mes | uf^grupo_cnae_fiscal_principal, base))

NOTE: 187 fixed-effect singletons were removed (187 observations).



OLS estimation, Dep. Var.: log_total_tpv_credit
Observations: 1,044,087
Fixed-effects: uf^grupo_cnae_fiscal_principal: 1,621
Standard-errors: IID 
                         Estimate Std. Error   t value  Pr(>|t|)    
log_total_tpv_credit_lag 0.848807   0.000556 1526.1837 < 2.2e-16 ***
log_total_tpv_credit_mun 0.010176   0.000425   23.9383 < 2.2e-16 ***
media_cnpjs_por_socio    0.013350   0.000321   41.5529 < 2.2e-16 ***
qtd_socios_no_mes        0.055571   0.001157   48.0154 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.957766     Adj. R2: 0.733561
                 Within R2: 0.712037

In [27]:
summary(feols(log_total_tpv_credit ~ log_total_tpv_credit_lag + log_total_tpv_credit_mun + media_cnpjs_por_socio + qtd_socios_no_mes | municipio + grupo_cnae_fiscal_principal, base))

NOTE: 76/0 fixed-effect singletons were removed (76 observations).

The variable 'log_total_tpv_credit_mun' has been removed because of collinearity
(see $collin.var).



OLS estimation, Dep. Var.: log_total_tpv_credit
Observations: 1,044,198
Fixed-effects: municipio: 5,445,  grupo_cnae_fiscal_principal: 77
Standard-errors: IID 
                         Estimate Std. Error   t value  Pr(>|t|)    
log_total_tpv_credit_lag 0.847679   0.000557 1522.6920 < 2.2e-16 ***
media_cnpjs_por_socio    0.013432   0.000320   41.9695 < 2.2e-16 ***
qtd_socios_no_mes        0.056740   0.001157   49.0454 < 2.2e-16 ***
... 1 variable was removed because of collinearity
(log_total_tpv_credit_mun)
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.953018     Adj. R2: 0.735238
                 Within R2: 0.705081